In [8]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import optuna
from tqdm import tqdm

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [9]:
class PreloadedImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.data = []
        self.labels = []
        transform = transform or transforms.ToTensor()

        for label_dir in os.listdir(root_dir):
            label_path = os.path.join(root_dir, label_dir)
            if not os.path.isdir(label_path):
                continue
            label = 1 if label_dir == "good" else 0
            for fname in tqdm(os.listdir(label_path)):
                if fname.lower().endswith(('.png', '.jpg', '.jpeg')):
                    path = os.path.join(label_path, fname)
                    img = Image.open(path).convert("L")
                    img_tensor = transform(img).to(device)
                    self.data.append(img_tensor)
                    self.labels.append(torch.tensor(label, dtype=torch.float32, device=device))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]


In [4]:
transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
])

In [5]:
class FlexibleCNN(nn.Module):
    def __init__(self, conv_layers=2, dense_layers=1, n_filters=16, hidden_dim=128, dropout_rate=0.5):
        super().__init__()
        layers = []
        in_channels = 1
        for _ in range(conv_layers):
            layers += [
                nn.Conv2d(in_channels, n_filters, 3, padding=1),
                nn.ReLU(),
                nn.MaxPool2d(2)
            ]
            in_channels = n_filters
            n_filters *= 2 
        self.features = nn.Sequential(*layers)

        reduced_size = 512 // (2 ** conv_layers)
        flatten_size = in_channels * reduced_size * reduced_size

        dense = [nn.Flatten()]
        for _ in range(dense_layers):
            dense.append(nn.Linear(flatten_size, hidden_dim))
            dense.append(nn.ReLU())
            dense.append(nn.Dropout(dropout_rate))
            flatten_size = hidden_dim

        dense.append(nn.Linear(flatten_size, 1))
        dense.append(nn.Sigmoid())
        self.classifier = nn.Sequential(*dense)

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x.squeeze(1)


In [10]:
full_dataset = PreloadedImageDataset('../data/', transform=transform)
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

100%|██████████| 280/280 [00:03<00:00, 73.41it/s]


In [ ]:
def train_model(model, loader, optimizer, criterion):
    model.train()
    for images, labels in loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

In [ ]:
def evaluate_model(model, loader):
    model.eval()
    all_labels, all_preds = [], []
    with torch.no_grad():
        for images, labels in loader:
            outputs = model(images)
            preds = (outputs > 0.5).int().cpu()
            all_labels.extend(labels.cpu())
            all_preds.extend(preds)
    acc = accuracy_score(all_labels, all_preds)
    prec = precision_score(all_labels, all_preds)
    rec = recall_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    return acc, prec, rec, f1

In [ ]:
def objective(trial):
    # Hiperparâmetros a serem otimizados
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32])
    lr = trial.suggest_float("lr", 1e-5, 1e-3, log=True)
    n_filters = trial.suggest_categorical("n_filters", [8, 16, 32])
    dropout = trial.suggest_float("dropout", 0.2, 0.7)
    conv_layers = trial.suggest_int("conv_layers", 1, 4)
    dense_layers = trial.suggest_int("dense_layers", 1, 3)
    hidden_dim = trial.suggest_categorical("hidden_dim", [64, 128, 256])

    model = FlexibleCNN(conv_layers=conv_layers, dense_layers=dense_layers,
                        n_filters=n_filters, hidden_dim=hidden_dim, dropout_rate=dropout).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCELoss()
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    for epoch in range(3):  # curtos para otimização
        train_model(model, train_loader, optimizer, criterion)

    acc, _, _, _ = evaluate_model(model, test_loader)
    return acc


: 

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

[I 2025-07-27 19:59:18,233] A new study created in memory with name: no-name-1e1435f1-d6fd-4c5f-823a-6cdc0be7940e


In [ ]:
best = study.best_params
model = FlexibleCNN(conv_layers=best["conv_layers"], dense_layers=best["dense_layers"],
                    n_filters=best["n_filters"], hidden_dim=best["hidden_dim"],
                    dropout_rate=best["dropout"]).to(device)
optimizer = optim.Adam(model.parameters(), lr=best["lr"])
criterion = nn.BCELoss()
train_loader = DataLoader(train_dataset, batch_size=best["batch_size"], shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=best["batch_size"])

In [ ]:
for epoch in range(10):
    train_model(model, train_loader, optimizer, criterion)

In [ ]:
acc, prec, rec, f1 = evaluate_model(model, test_loader)
print(f"Acurácia: {acc:.4f} | Precisão: {prec:.4f} | Recall: {rec:.4f} | F1: {f1:.4f}")

In [ ]:
metrics = ['Acurácia', 'Precisão', 'Recall', 'F1']
values = [acc, prec, rec, f1]
plt.bar(metrics, values)
plt.title("Desempenho Final")
plt.ylim(0, 1)
plt.savefig("metricas_final.png")
plt.show()

In [ ]:
torch.save(model.state_dict(), "cnn.pt")